In [1]:
# not autoreloading - raises TypeError in super().__init__() calls for models
from utils import *

from tqdm import tqdm
from pprint import pprint
import json
import _pickle as pkl
from time import time

In [2]:
glove = setup_glove()
print(glove.vectors.size())
VOCAB_SIZE = glove.vectors.size()[0]
with open('../data/data.json', 'r') as f:
    data = json.load(f)

torch.Size([400004, 50])


In [3]:
idx = 5
example_X = (data['X_train'][idx])
example_y = (data['y_train'][idx])
print("ID:", example_X[0])
print("Context:", example_X[1])
print("Question:", example_X[2])
print("Answer Span:", example_y)
print("Answer:", example_X[3])
X, y = make_data([example_X], [example_y])
pprint(y)
print(get_answer_span(y[0], example_X[1]))
num_ex = 100
X_pass, y_pass = make_data(data['X_train'][:num_ex], data['y_train'][:num_ex])

ID: 56ce602faab44d1400b88713
Context: A solar chimney (or thermal chimney, in this context) is a passive solar ventilation system composed of a vertical shaft connecting the interior and exterior of a building. As the chimney warms, the air inside is heated causing an updraft that pulls air through the building. Performance can be improved by using glazing and thermal mass materials in a way that mimics greenhouses.
Question: What kind of system is a solar chimney?
Answer Span: [59, 84]
Answer: passive solar ventilation
[(11, 13)]
passive solar ventilation


In [4]:
from models import *

conf = {"vocab": glove.vectors,
    "learning_rate": 0.1,
        "epochs": 5,
        "hidden_size": 32,
       "batch_size": 20,
       "opt": "Adamax",
        "n_layers": 1}
model = ModelV2(conf)
print(model, model.lr, model.hidden_size, model.batch_size, model.opt)
model_name = "BiGRU_%s_%s_%s_%s"%(num_ex, model.epochs, model.hidden_size, model.n_layers)
print(model_name)

ModelV2(
  (encoder_c): Embedding(400004, 50)
  (encoder_q): Embedding(400004, 50)
  (gru_c): GRU(50, 32, bidirectional=True)
  (gru_q): GRU(50, 32, bidirectional=True)
  (lin_q): Linear(in_features=64, out_features=64, bias=True)
  (gru_op): GRU(192, 128, bidirectional=True)
  (decoder_start): Linear(in_features=128, out_features=600, bias=True)
  (decoder_end): Linear(in_features=128, out_features=600, bias=True)
) 0.1 32 20 <class 'torch.optim.adamax.Adamax'>
BiGRU_100_5_32_1


In [ ]:
tic = time()
res = model.fit(X_pass, y_pass)
toc = time()
print("took", toc-tic, "seconds")
torch.save(model, '../evaluation/models/%s'%model_name)

In [5]:
model = torch.load('../evaluation/models/%s'%model_name)
print(model)
print(model_name)

ModelV2(
  (encoder_c): Embedding(400004, 50)
  (encoder_q): Embedding(400004, 50)
  (gru_c): GRU(50, 32, bidirectional=True)
  (gru_q): GRU(50, 32, bidirectional=True)
  (lin_q): Linear(in_features=64, out_features=64, bias=True)
  (gru_op): GRU(192, 128, bidirectional=True)
  (decoder_start): Linear(in_features=128, out_features=600, bias=True)
  (decoder_end): Linear(in_features=128, out_features=600, bias=True)
)
BiGRU_100_5_32_1


In [ ]:
import matplotlib.pyplot as plt
plt.plot(list(range(len(res))), res)
plt.show()

In [6]:
num_test = 100
X_test_data, y_test_data = make_data(data['X_test'][:num_test], data['y_test'][:num_test])

In [7]:
dev_results = {}
seen_contexts = set()
seen_questions = set()
from hashlib import sha1
    
for i, (x, y) in enumerate(zip(X_test_data[:num_test], y_test_data[:num_test])):
    model.hidden_c, model.hidden_q, model.hidden_op = model.init_hidden(1), model.init_hidden(1), model.init_hidden_2(1)
    qid = data['X_test'][i][0]
    if qid not in seen_questions:
        seen_questions.add(qid)
    else:
        continue
    c = data['X_test'][i][1]
    context_h = sha1(c.encode('utf-8')).hexdigest()
    if context_h not in seen_contexts:
        print(c)
        print("-"*30)
        seen_contexts.add(context_h)
    q = data['X_test'][i][2]
    print("QUESTION: ", q)
    a = data['X_test'][i][3]
    print("ANSWER: ", a)
    res = model.predict([x], bs=1).data.tolist()[0]
    if res[0]>res[1]:
        res[0], res[1] = res[1], res[0]
    ans = get_answer_span(res, c)
    print("PRED_SPAN: ", res)
    print("PREDICTED: ", ans)
    dev_results[qid] = ans
    print("="*50)
with open('../data/run_%sdata_%sepochs.json'%(num_ex, model.epochs), 'w') as f:
    json.dump(dev_results, f)

Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.
------------------------------
QUESTION:  Which NFL team represented the AFC at Super Bowl 50?
ANSWER:  Denver Broncos
PRED_SPAN:  [6, 31]
PREDICTED:  football game to determine the champion of the National Football League (

PRED_SPAN:  [6, 31]
PREDICTED:  football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the
QUESTION:  What team was the NFC champion?
ANSWER:  Carolina Panthers
PRED_SPAN:  [6, 31]
PREDICTED:  football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the
QUESTION:  Who won Super Bowl 50?
ANSWER:  Denver Broncos
PRED_SPAN:  [6, 31]
PREDICTED:  football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the
QUESTION:  Super Bowl 50 determined the NFL champion for what season?
ANSWER:  2015
PRED_SPAN:  [6, 31]
PREDICTED:  football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AF